[Bag of Words Meets Bags of Popcorn](https://www.kaggle.com/c/word2vec-nlp-tutorial/data)
======

## Data Set

The labeled data set consists of 50,000 IMDB movie reviews, specially selected for sentiment analysis. The sentiment of reviews is binary, meaning the IMDB rating < 5 results in a sentiment score of 0, and rating >=7 have a sentiment score of 1. No individual movie has more than 30 reviews. The 25,000 review labeled training set does not include any of the same movies as the 25,000 review test set. In addition, there are another 50,000 IMDB reviews provided without any rating labels.

## File descriptions

labeledTrainData - The labeled training set. The file is tab-delimited and has a header row followed by 25,000 rows containing an id, sentiment, and text for each review.
## Data fields

* id - Unique ID of each review
* sentiment - Sentiment of the review; 1 for positive reviews and 0 for negative reviews
* review - Text of the review

## Objective
Objective of this dataset is base on **review** we predict **sentiment** (positive or negative) so X is **review** column and y is **sentiment** column

## 1. Load Dataset
we only forcus on "labeledTrainData.csv" file

Let's first of all have a look at the data.

[Click here to download dataset](https://s3-ap-southeast-1.amazonaws.com/ml101-khanhnguyen/week3/assignment/labeledTrainData.tsv)

In [1]:
# import thư viện
import numpy
import pandas as pd
import nltk
from nltk.corpus import stopwords


In [2]:
# Read dataset with extra params sep='\t', encoding="latin-1"
# load dữ liệu bằng lệnh read_csv của pandas, kèm theo tham số sep để đọc file .tsv và encoding
df = pd.read_csv("labeledTrainData.tsv", sep="\t",encoding="latin-1")

## 2. Preprocessing

In [4]:
# stop words
# download stopwords
nltk.download('stopwords')
# lấy các stopword tiếng anh
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MyPC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [31]:
# from collections import Counter
# vocab_reduced = Counter()

# def stop_word(vocab_reduced):
#     for row in df.review:
#         for word in row.split():
#             word = word.lower()
#             if word in stop:
#                 continue
#             vocab_reduced[word] += 1
#     return vocab_reduced


# vocab_reduced.most_common(20)

In [21]:
# Removing special characters and "trash"
import re
def preprocessor(text):
    # Remove HTML markup
#     xóa các phần tử chứa html
    text = re.sub('<[^>]*>', '', text)
    # Save emoticons for later appending
#     Lưu các đối tượng emoticons
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
#     Xóa các đối tượng không phải ký tự và emoticons
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))
    
    return text

In [25]:
# tokenizer and stemming
# tokenizer: to break down our twits in individual words
# stemming: reducing a word to its root
from nltk.stem import PorterStemmer
porter = PorterStemmer()
# Dùng PorterStemmer để chuyển các từ giống nhau nhưng cách viết skhacs nhau về cùng dạng
def tokenizer_porter(text):
    token = [porter.stem(word) for word in text.split()]
    return token
 

In [28]:
# split the dataset in train and test
from sklearn.model_selection import train_test_split
# Tách thành bộ train và test
# input là cột review
X = df['review']
# output là cột sentiment
y = df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

## 3. Create Model and Train 

Using **Pipeline** to concat **tfidf** step and **LogisticRegression** step

In [32]:
# Import Pipeline, LogisticRegression, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
# Tiền xử lý dữ liệu
# dùng TfidfVectorizer để tạo ra vector mặt định dữ liệu qua đây điều được trải qua các bước xử lý mặt định
tfidf = TfidfVectorizer(stop_words=stop,
                        tokenizer=tokenizer_porter,
                        preprocessor=preprocessor)

# Tạo Pipeline để cả bài toán đi theo một quy trình mặt định là xử lý và áp dụng model xử lý(khai báo)
clf = Pipeline([('vect', tfidf),
                ('clf', LogisticRegression(random_state=0))])
# Tiến hành fit để huấn luyện model theo dữ liệu ta muốn
clf.fit(X_train, y_train)

d:\virtualtest\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
d:\virtualtest\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2',
        preprocessor=<function prepr...e, penalty='l2', random_state=0, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))])

## 4. Evaluate Model

In [33]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# sử dụng các phương pháp tính độ lỗi của model để hiệu chỉnh
# Using Test dataset to evaluate model
prediction = clf.predict(X_test)
print(accuracy_score(y_test, prediction))
# confusion matrix
print(confusion_matrix(y_test, prediction))
# classification_report
print(classification_report(y_test, prediction))

0.8857333333333334
[[3292  489]
 [ 368 3351]]
              precision    recall  f1-score   support

           0       0.90      0.87      0.88      3781
           1       0.87      0.90      0.89      3719

   micro avg       0.89      0.89      0.89      7500
   macro avg       0.89      0.89      0.89      7500
weighted avg       0.89      0.89      0.89      7500



## 5. Export Model 

In [34]:
# Using pickle to export our trained model
import pickle
import os
# có thể thay data bằng đường dẫn vào thư mục muốn lưu
pickle.dump(clf, open(os.path.join('save_model', 'logisticRegression.pkl'), 'wb'), protocol=4)